In [2]:
import casadi as ca
import numpy as np
from mpc_model import Model

ref: 
1. An augmented lagrangian based algorithm for distributed nonconvex optimization
2. Distributed Optimization with ALADIN for Non-convex Optimal Control Problems

In [29]:
def integrator_stage_cost(f, l, t, x, xr, u, ur, lambda_, delta_t):
    '''
    This function calculates the integration of stage cost with RK4.
    '''

    k1 = f(t, x, u)
    k2 = f(t + delta_t / 2, x + delta_t / 2 * k1, u)
    k3 = f(t + delta_t / 2, x + delta_t / 2 * k2, u)
    k4 = f(t + delta_t, x + delta_t * k3, u)

    Q = 0
    k1_q = l(x, xr, u, ur, lambda_)
    k2_q = l(x + delta_t / 2 * k1, xr, u, ur, lambda_)
    k3_q = l(x + delta_t / 2 * k2, xr, u, ur, lambda_)
    k4_q = l(x + delta_t * k3, xr, u, ur, lambda_)
    Q = Q + delta_t / 6 * (k1_q + 2 * k2_q + 2 * k3_q + k4_q)
    return Q

In [30]:
def integrator_rk4(f, t, x, u, delta_t):
    """
    Runge-Kutta 4th order solver using casadi.

    Args:
        f: First order ODE in casadi function (Nx + Nt -> Nx).
        t: Current time.
        x: Current value.
        u: Current input.
        delta_t: Step length.
    Returns:
        x_next: Vector of next value in casadi DM
    """
    k1 = f(t, x, u)
    k2 = f(t + delta_t / 2, x + delta_t / 2 * k1, u)
    k3 = f(t + delta_t / 2, x + delta_t / 2 * k2, u)
    k4 = f(t + delta_t, x + delta_t * k3, u)
    x_next = x + delta_t / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

    return x_next

In [31]:
def ode(t, x, u):
    """
    Mobile robot
    """
    # Parameter configuratio

    dx1_dt = x[1]
    dx2_dt = u[0] 
    rhs = [dx1_dt,
           dx2_dt
           ]
    return ca.vertcat(*rhs)

### Exmaple
$\begin{aligned} \min _{u_{i}, i \in \mathcal{V}} & \sum_{i \in \mathcal{V}} \int_{0}^{T}-e^{\left(-x_{1, i}^{2}\right)}-e^{\left(-x_{2, i}^{2}\right)}+\frac{1}{2} u_{i}^{2} \mathrm{~d} \tau \\ \text { s.t. } &\left[\begin{array}{l}\dot{x}_{1, i} \\ \dot{x}_{2, i}\end{array}\right]=\left[\begin{array}{c}x_{2, i} \\ u_{i}\end{array}\right] \\ & 1=\sum_{i \in \mathcal{V}} x_{1, i}+x_{2, i}+u_{i} \end{aligned}$

With $T = 1s$ and $N = |\mathcal{V}|=10$.

In [32]:
b = 1
Ci = np.array([[1, 1]])
Di = 1
rho = 1

Sigma_xi = ca.diag([1,1])
Sigma_ui = ca.diag([1])

In [33]:
Nt = 1
N_lambda, Nx = np.shape(Ci)
Nu = 1


delta_t = 0.1
N_pred = 10
# N_sim = 10

t_SX = ca.SX.sym("t_SX", Nt)
x_SX = ca.SX.sym("x_SX", Nx)
u_SX = ca.SX.sym("u_SX", Nu)
lambda_SX = ca.SX.sym("lambda_SX", N_lambda)

ode(t_SX, x_SX, u_SX)

xr_SX = ca.SX.sym("xr_SX", Nx)
ur_SX = ca.SX.sym("ur_SX", Nu)

# Cost function to calculate the local solution
stage_cost =  -ca.exp(-x_SX[0] **2) -ca.exp(-x_SX[1] **2) + 1/2 * u_SX[0]**2  #  Lagrange term
stage_cost += lambda_SX.T @ (Ci @ x_SX + Di @ u_SX) + rho / 2 * (x_SX - xr_SX).T @ Sigma_xi @ (x_SX - xr_SX) + rho / 2 * (u_SX - ur_SX).T @ Sigma_ui @ (u_SX - ur_SX)
terminal_cost = 0   #  Mayer term
stage_cost_func = ca.Function("stage_cost_func",[x_SX, xr_SX, u_SX, ur_SX, lambda_SX], [stage_cost])
terminal_cost_func = ca.Function("terminal_cost_func",[x_SX, xr_SX], [terminal_cost])

ode_func = ca.Function("ode_func", [t_SX, x_SX, u_SX], [ode(t_SX, x_SX, u_SX)])
ode_dis_model = integrator_rk4(ode_func, t_SX, x_SX, u_SX, delta_t)
ode_dis_model_func = ca.Function("ode_dis_model_func",[t_SX, x_SX, u_SX],[ode_dis_model])
stage_cost_dis = integrator_stage_cost(ode_func, stage_cost_func, t_SX, x_SX, xr_SX, u_SX, ur_SX, lambda_SX, delta_t)
stage_cost_dis_func = ca.Function("stage_cost_dis_func",[x_SX, xr_SX, u_SX, ur_SX, lambda_SX],[stage_cost_dis])

### Example 1
$\min _{x} x_{1} \cdot x_{2} \quad$ s.t. $\quad x_{1}-x_{2}=0$

analytical solution of $y$ is: $y=\left(\begin{array}{c}-2 \\ 2\end{array}\right) \lambda$

Numerical problem of ADMM with divergent $\lambda^{+} = - 2 \lambda$

### Subproblem formulation
$\min _{y_{i}} f_{i}\left(y_{i}\right)+\lambda^{\top} A_{i} y_{i}+\frac{\rho}{2}\left\|y_{i}-x_{i}\right\|_{\Sigma_{i}}^{2} \quad$ s.t. $\quad h_{i}\left(y_{i}\right) \leq 0 \mid \kappa_{i}$

In [687]:
def create_subproblem(fi_func, Ai, rho, hi_func):
    N_lambda, N_yi = np.shape(Ai)
    yi = ca.SX.sym("yi",N_yi)
    xi = ca.SX.sym("xi",N_yi)
    sigma_i = ca.SX.sym('sigma_i',N_yi,N_yi)
    lambda_ = ca.SX.sym("lambda",N_lambda)
    
    fi = fi_func(yi) + lambda_.T @ Ai @ yi + rho/2 * (yi - xi).T @ sigma_i @ (yi - xi)
    p = ca.vertcat(lambda_, xi, ca.reshape(sigma_i, -1,1))
    g = hi_func(yi)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':yi, 'g':g, 'f':fi, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver

In [421]:
# N =3
# zero_row = ca.DM.zeros(1,3)


# A = ca.DM([[1,2,3],[4,5,6],[7,8,9]])
# A[1,:] = zero_row
# A

In [422]:
# x = ca.SX.sym("x",3)
# hi = ca.vertcat(x[1]+x[2],x[0]+x[2])
# hi_func = ca.Function("hi_func",[x],[hi])
# hi_jac = ca.jacobian(hi,x)
# hi_jac_func = ca.Function('hi_jac_func',[x], [hi_jac])

# y = ca.DM([1,-1,1])
# hi_jac_approx = constraint_jac_approx(y,hi_func,hi_jac_func)
# hi_jac_real = hi_jac_func(y)

# print(hi_jac_approx,hi_jac_real)

In [423]:
def constraint_jac_approx(yi, hi_func, hi_jac_func):
    constraint_res = hi_func(yi)    #  Residue
    Nh = np.shape(constraint_res)[0]
    Ny = np.shape(yi)[0]
    zero_row = ca.DM.zeros(1,Ny)
    hi_jac = hi_jac_func(yi)
    for i in range(Nh):
        if constraint_res[i] != 0:    #  TODO: deal with small value
            hi_jac[i,:] = zero_row
    return hi_jac

In [925]:
def modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i):
    return fi_grad + (hi_jac_real - hi_jac_approx).T @ kappa_i
    

In [952]:
def create_QP_problem(A_list, b,  mu, N_hi_list):
    N = len(A_list)
    N_lambda = np.shape(A_list[0])[0]
    
    s = ca.SX.sym("s", N_lambda)
    lambda_ = ca.SX.sym("lambda_", N_lambda)
    
    delta_yi_list = []
    fkh_hess_col_list = [] 
    modiefied_grad_col_list = []
    Ci_col_list = []
    
    yi_list = []
    obj = 0
    sigma_Ai = 0

    g = []
    for i in range(N):
        Ai = A_list[i]
        N_delta_yi = np.shape(Ai)[1]
        Hi = ca.SX.sym("Hi" + str(i), N_delta_yi, N_delta_yi)
        gi = ca.SX.sym("gi" + str(i), N_delta_yi)
        yi = ca.SX.sym("yi" + str(i), N_delta_yi)
        Ci = ca.SX.sym("Ci" + str(i), N_hi_list[i], N_delta_yi)
        
        fkh_hess_col_list += [ca.reshape(Hi, -1, 1)]
        modiefied_grad_col_list += [ca.reshape(gi, -1, 1)]
        yi_list += [yi]
        
        delta_yi = ca.SX.sym("delta_yi" + str(i),N_delta_yi)
        delta_yi_list += [delta_yi]
    
        obj += 1/2 * delta_yi.T @ Hi @ delta_yi + gi.T @ delta_yi
        sigma_Ai += Ai @ (yi + delta_yi)
        
        Ci_col_list += [ca.reshape(Ci, -1, 1)]
        g += [Ci @ delta_yi]
    obj += lambda_.T @ s + mu/2 * s.T @ s
    x = ca.vertcat(*delta_yi_list, s)
    p = ca.vertcat(lambda_, *(fkh_hess_col_list + modiefied_grad_col_list + yi_list + Ci_col_list))

    g += [ sigma_Ai - b - s ]
    g = ca.vertcat(*g)
    # Define proximal solver
    solver_opt = {}
    solver_opt['print_time'] = False
    solver_opt['ipopt'] = {
        'max_iter': 500,
        'print_level': 1,
        'acceptable_tol': 1e-6,
        'acceptable_obj_change_tol': 1e-6
    }

    nlp = {'x':x, 'g':g, 'f':obj, 'p': p}
#     print(nlp)
    solver = ca.nlpsol('solver', 'ipopt', nlp, solver_opt)
    return solver    

In [426]:
x = ca.SX.sym("x",2)
f = x[0] * x[1]
ca.hessian(f, x)



(SX(@1=1, 
 [[00, @1], 
  [@1, 00]]),
 SX([x_1, x_0]))

In [469]:
eps = 1e-5
# rho = 0.75
rho = 1e5
N_itermax = 50
A_list = []
sigma_list = []
fi_func_list = []
hi_func_list = []
N_hi_list = []

A = ca.DM([[1,-1]])
A_list += [A]
N = len(A_list)
b = ca.DM([0])

sigma_i = ca.diag([1,1])
sigma_list += [sigma_i]

Nx = 2
x = ca.SX.sym("x",Nx)
fi = x[0] * x[1]
fi_func = ca.Function("fi_func", [x], [fi])
fi_func_list += [fi_func]

fi_grad_func_list = []
fi_grad = ca.gradient(fi, x)
fi_grad_func = ca.Function("fi_grad_func", [x], [fi_grad])
fi_grad_func_list += [fi_grad_func]


# hi = ca.vertcat(x[0]+x[1]) # To be modified
hi = ca.vertcat(x[0]+x[1] - 2) # To be modified
hi_func = ca.Function("hi_func", [x], [hi])
Nhi = np.shape(hi)[0]
N_hi_list = [Nhi]
hi_func_list = [hi_func]

kappa_i = ca.SX.sym("kappa_i",Nhi)


hi_jac_func_list = []
fkh_hess_func_list = []
hi_jac = ca.jacobian(hi,x)
hi_jac_func = ca.Function("hi_jac_func",[x],[hi_jac])
hi_jac_func_list +=  [hi_jac_func]
fkh_i = fi + kappa_i.T @ hi
fkh_hess_i = ca.hessian(fkh_i, x)[0]
fkh_hess_i_func = ca.Function("fkh_hess_i_func", [x, kappa_i], [fkh_hess_i])
fkh_hess_func_list += [fkh_hess_i_func]

In [475]:
subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    sigma_i = sigma_list[i]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho, hi_func)]    
mu = 1e5
QP_solver = create_QP_problem(A_list, b,  mu, N_hi_list)
# Initial guess
delta_yi_list = []
sigma_i_list = []
xi_list = []
yi_list = []
lbhi_list = []
ubhi_list = []
lbx_list = []
ubx_list = []
Nx = 0
N_hi_sum = 0
for i in range(N):
    Ai = A_list[i]
    N_lambda, N_xi = np.shape(Ai)
    Nx += N_xi
    N_hi = N_hi_list[i]
    N_hi_sum += N_hi
#     xi = np.random.randn(N_xi,1).flatten().tolist()
    xi = ca.DM.zeros(N_xi,1).full().flatten().tolist()
    xi_list += [xi]
    sigma_i_list += [ca.diag([1] * N_xi)]
    
    lbhi_list += [[-ca.inf] * N_hi]
    ubhi_list += [[0] * N_hi]
    yi_list += [[0] * N_xi]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
lambda_ = np.random.randn(N_lambda,1)
lambda_ = ca.DM.zeros(N_lambda,1)
print(lambda_)
s_list = [0] * N_lambda
delta_yi_list = sum(yi_list,[])
    
nl_sub = {}



nl_qp = {}
nl_qp['lbg'] = [0] * (N_hi_sum + N_lambda)
nl_qp['ubg'] = [0] * (N_hi_sum + N_lambda)
nl_qp['lbx'] = sum(lbx_list,[]) + [-np.inf] * N_lambda    # delta_y and s lower bound
nl_qp['ubx'] = sum(ubx_list,[]) + [+np.inf] * N_lambda    # delta_y and s upper bound

# Track solution
yi_sol_list = []
delta_y_sol_list = []
lambda_list = []
x_sol_list = []
x_sol_list += [xi_list.copy()]


# delta_y_sol_list += [sum(xi_list,[])]
lambda_list += lambda_.full().flatten().tolist()
for i in range(N_itermax):
    sum_Ay = 0
    kappa_list = []
    # Step 1: solve the subproblem      
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        sigma_i = sigma_i_list[j]
        nl_sub['lbg'] = lbhi_list[j]
        nl_sub['ubg'] = ubhi_list[j]    
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_list + xi_list[j] + ca.reshape(sigma_i, -1, 1).full().flatten().tolist()

        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
#         print(yi_list)
        yi_sol_list += [yi_list[j].copy()]
        kappa_i = yi_sol['lam_g']
        kappa_list += [kappa_i]
        
        sum_Ay += Ai @ yi_sol['x']

    # Step 2: Check if the tolerance satisfied
    #TODO: modify
    N_flag = 0
    for j in range(N):
        if rho * ca.norm_1( sigma_i_list[j] @ ca.DM(yi_list[j])) <= eps:
            N_flag += 1
    if ca.norm_1(sum_Ay - b) <= eps and N_flag == N:
        break
    # Step3: update Jacobian approximations, calculate the modified gradient, and update Hessian
    Ci_list = []    #  constraint Jacobian
    g_list = []    #  modified gradient
    H_list = []    #  Hessian
    for j in range(N):
        # 3.1 Choose Jacobian approximations
        yi = yi_list[j]
        hi_func = hi_func_list[j]
        hi_jac_func = hi_jac_func_list[j]
        fkh_hess_func = fkh_hess_func_list[j]
        hi = hi_func(yi)
        kappa_i = kappa_list[j]
        fi_grad = fi_grad_func_list[j](yi)
        hi_jac_real = hi_jac_func(yi)
        
        hi_jac_approx = constraint_jac_approx(yi, hi_func, hi_jac_func)
        Ci_list += [ca.reshape(hi_jac_real, -1, 1)]
        gi = modified_grad(yi,fi_grad, hi_jac_approx, hi_jac_real, kappa_i)
        g_list += [ca.reshape(gi, -1, 1)]
        
        Hi = fkh_hess_func(x, kappa_i)
        H_list += [ca.reshape(Hi, -1, 1)]
#         print("hi", hi, "kappa_i",kappa_i, "fi_grad",fi_grad, "hi_jac_real",hi_jac_real, "hi_jac_approx",hi_jac_approx, "gi",gi, "Hi" ,Hi)
    # Step 4: Solve QP problem
    nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess
    
    H_para = ca.vertcat(*H_list)
    modiefied_grad = ca.vertcat(*g_list)
    y = ca.vertcat(* sum(yi_list,[]))
    Ci = ca.vertcat(*Ci_list)
    lambda_ = ca.vertcat(lambda_list)
    p = ca.vertcat(lambda_, H_para, modiefied_grad, y, Ci)
    nl_qp['p'] = ca.DM(p)
#     print(nl_qp)
    QP_sol = QP_solver(**nl_qp)
    
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    # Step 5: Update x and lambda
    pos = 0
    
    delta_y = QP_sol['x'][0:Nx,:]
    QP_list = QP_sol['x'].full().flatten().tolist()
    lambda_QP = QP_sol['lam_g'][:N_lambda]
#     print("lambda_QP", lambda_QP)
    x = ca.DM(sum(xi_list,[]))
    y = ca.DM(sum(yi_list,[]))
    x_plus = x + alpha1 * (y - x) + alpha2 * delta_y
    for j in range(N):
        list_len = len(xi_list[j])
        xi_list[j] = x_plus[pos:pos+list_len].full().flatten().tolist()
       
    lambda_ = lambda_ + alpha3 * (lambda_QP - lambda_)
    lambda_list = lambda_.full().flatten().tolist()
    x_sol_list += xi_list.copy()

0


In [476]:
yi_sol_list

[[-1.2529141938832168e-14, -1.2529141938832168e-14]]

In [477]:
x_sol_list

[[[0.0, 0.0]]]

### Example 2
Case study of sensor network localization problem:

$f_{i}\left(x_{i}\right)=\frac{1}{4 \sigma_{i}^{2}}\left\|\chi_{i}-\eta_{i}\right\|_{2}^{2}+\frac{1}{4 \sigma_{i+1}^{2}}\left\|\zeta_{i}-\eta_{i+1}\right\|_{2}^{2}+\frac{1}{2 \bar{\sigma}_{i}^{2}}\left(\left\|\chi_{i}-\zeta_{i}\right\|_{2}-\bar{\eta}_{i}\right)^{2}$, where $\bar{\eta}_{i}=2 N \sin \left(\frac{\pi}{N}\right)+d_{i}$, $\eta_{i}=\left(\begin{array}{c}N \cos \left(\frac{2 i \pi}{N}\right) \\ N \sin \left(\frac{2 i \pi}{N}\right)\end{array}\right)+e_{i}$ and $x_{i}=\left(\chi_{i}^{\top}, \zeta_{i}^{\top}\right)^{\top} \in \mathbb{R}^{4}$

$\sum_{i=1}^{N} A_{i} x_{i}=0$ with $A_{1}=\left(\begin{array}{cc}0 & I \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ \vdots & \vdots \\ 0 & 0 \\ -I & 0\end{array}\right), \quad A_{2}=\left(\begin{array}{cc}-I & 0 \\ 0 & I \\ 0 & 0 \\ 0 & 0 \\ \vdots & \vdots \\ 0 & 0 \\ 0 & 0\end{array}\right), \quad A_{2}=\left(\begin{array}{cc}0 & 0 \\ 0 & 0 \\ -I & 0 \\ 0 & I \\ \vdots & \vdots \\ 0 & 0 \\ 0 & 0\end{array}\right), \ldots, A_{N}=\left(\begin{array}{cc}0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ \vdots & \vdots \\ -I & 0 \\ 0 & I\end{array}\right) .$

### Remark
1. initial guess of y cannot all be zero

In [1434]:
np.random.seed(2)
N = 3 # Should test with 25000
Nx = 4
eps = 1e-5
sigma_i = 10
sigma_bar_i = 10
# rho = 0.75
rho = 1
N_itermax = 15

# Define A matrix
A_list = []
NA_col = int(N * Nx / 2)
I = ca.diag([1,1])
for i in range(N):
    A = ca.DM.zeros(NA_col, Nx)
    if i == 0:
        A[NA_col-2: , :2] = -I
    else:
        A[(i-1)*2: (i-1)*2+2, :2] = -I        
    A[i*2:i*2+2,Nx-2:] = I
    A_list += [A]
# Define b
b = ca.DM.zeros(NA_col,1)

In [1435]:
# # Define Sigma_i matrix
# Sigma_list = []
# for i in range(N):
#     Sigma = ca.diag([1,1,1,1])
#     Sigma_list += [Sigma]

In [1436]:
# Define parameter
eta_list = []
eta_bar_list = []

for i in range(N):
    eta = np.array([[N * np.cos(2*(i+1) * np.pi / N)],[N * np.sin(2*(i+1) * np.pi / N)]]) + sigma_i * np.random.randn(2,1)
    eta_list +=  [ca.DM(eta)]
    
    eta_bar = 2 * N * np.sin(np.pi / N) + sigma_i * np.random.randn(1)
    eta_bar_list += [ca.DM(eta_bar)]
    

$x_{i}=\left(\chi_{i}^{\top}, \zeta_{i}^{\top}\right)^{\top} \in \mathbb{R}^{4}$

In [1437]:
# Define obejective function
fi_list = []
fi_func_list = []
x = ca.SX.sym("x",Nx)    #  $x_{i}=\left(\chi_{i}^{\top}, \zeta_{i}^{\top}\right)^{\top} \in \mathbb{R}^{4}$
for i in range(N):
    if i == N-1:
        fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[0]).T @ (x[2:] - eta_list[0]) + 1 / (2 * sigma_bar_i**2) * ( ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i] )**2
    else:
        fi = 1 / (4 * sigma_i ** 2) * (x[0:2] - eta_list[i]).T @ (x[0:2] - eta_list[i]) + 1 / (4 * sigma_i**2) * (x[2:] - eta_list[i+1]).T @ (x[2:] - eta_list[i+1]) + 1 / (2 * sigma_bar_i**2) * (ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i])**2
    fi_list += [fi]
    fi_func = ca.Function("fi_func", [x], [fi])
    fi_func_list += [fi_func]

In [1438]:
# Define gradtient function
fi_grad_list = []
fi_grad_func_list = []
for i in range(N):
    fi_grad = ca.gradient(fi_list[i], x)
    fi_grad_list += [fi_grad]
    fi_grad_func = ca.Function("fi_grad_func", [x], [fi_grad])
    fi_grad_func_list += [fi_grad_func]

In [1439]:
# Define inequality constraints
hi_list = []
hi_func_list = []
Nhi_list = []
for i in range(N):
    hi = (ca.norm_2(x[0:2] - x[2:]) - eta_bar_list[i])**2 - sigma_bar_i ** 2
    hi_list += [hi]
    hi_func = ca.Function("hi_func", [x], [hi])
    hi_func_list += [hi_func]
    # Deal with the number of inequality constraints for each i.
    Nhi = np.shape(hi)[0]
    Nhi_list += [Nhi]

In [1440]:
# Define approximate jacobian, real jacobian and Hessian.
kappa_i_list = []
hi_jac_list = []
hi_jac_func_list = []
fkh_hess_i_list = []
fkh_hess_func_list = []
for i in range(N):
    # Kappa
    kappa_i = ca.SX.sym("kappa_i",Nhi)
    kappa_i_list += [kappa_i]
    # Jacobian function
    hi_jac = ca.jacobian(hi_list[i],x)
    hi_jac_list += [hi_jac]
    hi_jac_func = ca.Function("hi_jac_func",[x],[hi_jac])
    hi_jac_func_list +=  [hi_jac_func]
    # Hessian fucntion
    fi = fi_list[i]
    hi = hi_list[i]
    fkh_i = fi + kappa_i_list[i].T @ hi
    fkh_hess_i = ca.hessian(fkh_i, x)[0]
    fkh_hess_i_func = ca.Function("fkh_hess_i_func", [x, kappa_i], [fkh_hess_i])
    fkh_hess_i_list += [fkh_hess_i]
    fkh_hess_func_list += [fkh_hess_i_func]

In [1441]:
subsolver_list = []
# Define subproblem solvers
for i in range(N):
    Ai = A_list[i]
    fi_func = fi_func_list[i]
    hi_func = hi_func_list[i]
    subsolver_list += [create_subproblem(fi_func, Ai, rho, hi_func)]

In [1442]:
# Define QP problem
mu = 10
QP_solver = create_QP_problem(A_list, b,  mu, Nhi_list)

In [1443]:
eta_list[i].shape

(2, 1)

In [1444]:
eta_list

[DM([-5.66758, 2.03541]), DM([14.9027, -20.5324]), DM([8.02881, -12.4529])]

In [1445]:
# Initial guess
delta_yi_list = []
Sigma_i_list = []
xi_list = []
yi_list = []
lbhi_list = []
ubhi_list = []
lbx_list = []
ubx_list = []
Nx = 0
Nhi_sum = 0
for i in range(N):
    Ai = A_list[i]
    N_lambda, N_xi = np.shape(Ai)
    Nx += N_xi
    Nhi = Nhi_list[i]
    Nhi_sum += Nhi
#     xi = np.random.randn(N_xi,1).flatten().tolist()
# #     xi = ca.DM.zeros(N_xi,1).full().flatten().tolist()
    if i == N-1:
        xi = ca.vertcat(eta_list[N-1],eta_list[0]).full().flatten().tolist()
    else:
        xi = ca.vertcat(eta_list[i],eta_list[i+1]).full().flatten().tolist()
    xi_list += [xi]

    print(N_xi)
    Sigma_i_list += [ca.diag([1] * N_xi)]
    
    lbhi_list += [[-ca.inf] * Nhi]
    ubhi_list += [[0] * Nhi]
    yi = np.random.randn(N_xi,1).flatten().tolist()
    yi_list += [yi]   
#     yi_list += [[0] * N_xi]
    lbx_list += [[-ca.inf] * N_xi]
    ubx_list += [[ca.inf] * N_xi]
lambda_ = np.random.randn(N_lambda,1)
lambda_ = ca.DM(lambda_)
# lambda_ = ca.DM.zeros(N_lambda,1)
s_list = [0] * N_lambda
delta_yi_list = sum(yi_list,[])

4
4
4


In [1446]:
# Define solver
nl_sub = {}

nl_qp = {}
nl_qp['lbg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['ubg'] = [0] * (Nhi_sum + N_lambda)
nl_qp['lbx'] = sum(lbx_list,[]) + [-np.inf] * N_lambda    # delta_y and s lower bound
nl_qp['ubx'] = sum(ubx_list,[]) + [+np.inf] * N_lambda    # delta_y and s upper bound

In [1447]:
# Track solution
yi_sol_list = []
delta_y_sol_list = []
lambda_list = []
x_sol_list = []
x_sol_list += [xi_list.copy()]
lambda_list += lambda_.full().flatten().tolist()

In [1448]:
# solve problem

for i in range(N_itermax):
    sum_Ay = 0
    kappa_sol_list = []
    # Step 1: solve the subproblem      
    for j in range(N):
        Ai = A_list[j]
        N_lambda_i, N_xi = np.shape(Ai)
        Sigma_i = Sigma_i_list[j]
        nl_sub['lbg'] = lbhi_list[j]
        nl_sub['ubg'] = ubhi_list[j]    
        nl_sub['x0'] = yi_list[j]
        nl_sub['lbx'] = lbx_list[j]
        nl_sub['ubx'] = ubx_list[j]
        nl_sub['p'] = lambda_list + xi_list[j] + ca.reshape(Sigma_i, -1, 1).full().flatten().tolist()

        solver_subproblem = subsolver_list[j]
        yi_sol = solver_subproblem(**nl_sub)
        yi_list[j] = yi_sol['x'].full().flatten().tolist()
#         print(yi_sol['x'])
        yi_sol_list += [yi_list[j].copy()]
        kappa_i_sol = yi_sol['lam_g']
        kappa_sol_list += [kappa_i_sol]
        
        sum_Ay += Ai @ yi_sol['x']

    # Step 2: Check if the tolerance satisfied
    #TODO: modify
#     N_flag = 0
#     for j in range(N):
#         if rho * ca.norm_1( Sigma_i_list[j] @ ca.DM(yi_list[j])) <= eps:
#             N_flag += 1
#     if ca.norm_1(sum_Ay - b) <= eps and N_flag == N:
#         break
    # Step3: update Jacobian approximations, calculate the modified gradient, and update Hessian
    Ci_list = []    #  constraint Jacobian
    g_list = []    #  modified gradient
    H_list = []    #  Hessian
    for j in range(N):
        # 3.1 Choose Jacobian approximations
        yi = yi_list[j]
        hi_func = hi_func_list[j]
        hi_jac_func = hi_jac_func_list[j]
        fkh_hess_func = fkh_hess_func_list[j]
        hi = hi_func(yi)
        kappa_i_sol = kappa_sol_list[j]
        fi_grad = fi_grad_func_list[j](yi)
        hi_jac_real = hi_jac_func(yi)
        
        hi_jac_approx = constraint_jac_approx(yi, hi_func, hi_jac_func)
        Ci_list += [ca.reshape(hi_jac_real, -1, 1)]
        gi = modified_grad(fi_grad, hi_jac_approx, hi_jac_real, kappa_i_sol)
        g_list += [ca.reshape(gi, -1, 1)]
        
        Hi = fkh_hess_func(yi, kappa_i_sol)
        H_list += [ca.reshape(Hi, -1, 1)]
#         print("hi", hi, "kappa_i_sol",kappa_i_sol, "fi_grad",fi_grad, "hi_jac_real",hi_jac_real, "hi_jac_approx",hi_jac_approx, "gi",gi, "Hi" ,Hi)
    # Step 4: Solve QP problem
    nl_qp['x0'] = delta_yi_list + s_list    #  Initial guess
    
    H_para = ca.vertcat(*H_list)
    modified_grad_value = ca.vertcat(*g_list)
    y = ca.vertcat(* sum(yi_list,[]))
    Ci = ca.vertcat(*Ci_list)
    lambda_ = ca.vertcat(lambda_list)
    p = ca.vertcat(lambda_, H_para, modified_grad_value, y, Ci)
    nl_qp['p'] = ca.DM(p)
    print(lambda_)
    print(H_para)
    print(modified_grad_value)
    print(y)
    print(Ci)
    QP_sol = QP_solver(**nl_qp)
#     print(QP_sol)
    alpha1 = 1
    alpha2 = 1
    alpha3 = 1
    # Step 5: Update x and lambda
    pos = 0
#     print(QP_sol['x'][0:Nx,:])
    delta_y = QP_sol['x'][0:Nx,:]
    QP_list = QP_sol['x'].full().flatten().tolist()
    lambda_QP = QP_sol['lam_g'][:N_lambda]
#     print("lambda_QP", lambda_QP)
    x = ca.DM(sum(xi_list,[]))
    y = ca.DM(sum(yi_list,[]))
#     print(delta_y)
#     print(y,x)
    x_plus = x + alpha1 * (y - x) + alpha2 * delta_y
    for j in range(N):
        list_len = len(xi_list[j])
        xi_list[j] = x_plus[pos:pos+list_len].full().flatten().tolist()
        pos = pos+list_len
#     print(xi_list)
#     print(lambda_QP)
    lambda_ = lambda_ + alpha3 * (lambda_QP - lambda_)
    lambda_list = lambda_.full().flatten().tolist()
    x_sol_list += [xi_list.copy()]

[-0.156434, 0.25657, -0.988779, -0.338822, -0.236184, -0.637655]
[0.104774, 2.06914, -0.0997741, -2.06914, 2.06914, 1464.85, -2.06914, -1464.85, -0.0997741, -2.06914, 0.104774, 2.06914, -2.06914, -1464.85, 2.06914, 1464.85, 0.216684, 0.0358986, -0.211684, -0.0358986, 0.0358986, 0.190815, -0.0358986, -0.185815, -0.211684, -0.0358986, 0.216684, 0.0358986, -0.0358986, -0.185815, 0.0358986, 0.190815, 1.31245, 0.436733, -1.30745, -0.436733, 0.436733, 1.12381, -0.436733, -1.11881, -1.30745, -0.436733, 1.31245, 0.436733, -0.436733, -1.11881, 0.436733, 1.12381]
[1.61701, -0.0608559, -1.61741, 0.056407, 0.917874, -1.30619, -0.913733, 1.30915, 4.7841, -5.93538, -4.78784, 5.93686]
[4.57842, -9.6934, 4.57735, -9.6934, 13.8284, -18.9697, 9.93133, -13.4232, 2.25594, -6.85633, -0.643552, -3.2638]
[32.3337, -0.0456753, -32.3337, 0.0456753, 11.498, -16.3645, -11.498, 16.3645, 12.5611, -15.5634, -12.5611, 15.5634]
[-0.731783, -1.96358, -2.26167, -22.0053, 30.63, -21.955]
[3.02082, 4.0668, -3.01582, -4.0

[-0.0707569, -0.0985687, -0.117629, 1.03033, -1.15592, 1.08488]
[0.339313, -0.301148, -0.334313, 0.301148, -0.301148, 0.717476, 0.301148, -0.712476, -0.334313, 0.301148, 0.339313, -0.301148, 0.301148, -0.712476, -0.301148, 0.717476, 0.18946, -0.02226, -0.18446, 0.02226, -0.02226, 0.261459, 0.02226, -0.256459, -0.18446, 0.02226, 0.18946, -0.02226, 0.02226, -0.256459, -0.02226, 0.261459, 0.0375554, -0.0128248, -0.0325554, 0.0128248, -0.0128248, 0.0507032, 0.0128248, -0.0457032, -0.0325554, 0.0128248, 0.0375554, -0.0128248, 0.0128248, -0.0457032, -0.0128248, 0.0507032]
[-2.90078, -1.68874, 2.88147, 1.67798, -1.77399, -0.446986, 1.71994, 0.495821, 0.223193, 0.15209, -0.149842, -0.190176]
[1.01794, -11.2464, 4.35613, -9.4007, 2.80096, -12.5356, 9.32143, -10.6825, 10.4853, -7.81359, 6.54608, -10.221]
[-34.9711, -19.3356, 34.9711, 19.3356, -19.2382, -5.46747, 19.2382, 5.46747, 17.0655, 10.4292, -17.0655, -10.4292]
[-0.358193, -0.625829, 0.535526, 9.61972, 5.28238, 9.65563]
[0.845735, -0.00038

In [1449]:
xi_list

[[4.232115964822362,
  -12.63130570892119,
  4.3736244607622154,
  -6.488365290727579],
 [4.367692839264673,
  -6.526995736765285,
  8.589231494451978,
  -11.811884571137064],
 [8.540961539568855,
  -11.778296080155304,
  4.424261264281203,
  -12.66296259286226]]

In [1450]:
x_sol_list

[[[-5.6675784740547055,
   2.035407939090022,
   14.902708084049886,
   -20.532432063301947],
  [14.902708084049886,
   -20.532432063301947,
   8.028814171580429,
   -12.452880866072316],
  [8.028814171580429,
   -12.452880866072316,
   -5.6675784740547055,
   2.035407939090022]],
 [[5.531490283393275,
   -10.950243191799423,
   5.530391830256668,
   -10.971162940244097],
  [7.715283167076178,
   -14.008502740830576,
   5.149305795608632,
   -7.526777022991431],
  [7.245929177943171,
   -10.617931048798178,
   3.355487308872387,
   -7.825193035768373]],
 [[6.429204901996378,
   -11.131723640010078,
   5.5693443048788644,
   -12.049574309164187],
  [5.673898659401664,
   -13.704497846887563,
   6.85960799347055,
   -6.940542538605743],
  [6.805974631043373,
   -10.593472643258371,
   3.189857072356725,
   -7.479999002644461]],
 [[4.344978465788078,
   -11.67930820948588,
   4.342861148409661,
   -11.677041348519715],
  [4.1669136956613535,
   -12.5017839228192,
   9.49344482402853,
   -

In [1452]:
eta_list

[DM([-5.66758, 2.03541]), DM([14.9027, -20.5324]), DM([8.02881, -12.4529])]